In [ ]:
from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np

from contact_graspnet.dataloading import OrigExampleData, YCBSimulationData
from contact_graspnet.utils.misc import get_root_dir
from contact_graspnet.preprocessing import Img2CamCoords, Depth2ImgPoints, ZClipper, UniversalPreprocessor, BinarySegmentationSample

In [ ]:

datasets = [
    OrigExampleData(get_root_dir() / "data" / "raw" / "orig_test_data", transform=BinarySegmentationSample(1.0))
    OrigExampleData(get_root_dir() / "data" / "raw" / "orig_test_data", transform=BinarySegmentationSample(2.0))
    OrigExampleData(get_root_dir() / "data" / "raw" / "orig_test_data", transform=BinarySegmentationSample(3.0))
    YCBSimulationData(Path.home() / "Documents" / "ycb_sim_data_1"),
    YCBSimulationData(Path.home() / "Documents" / "ycb_sim_data_2"),
    YCBSimulationData(Path.home() / "Documents" / "ycb_sim_data_3"),
]

In [ ]:
preprocessor = UniversalPreprocessor(
    depth2points_converter = Depth2ImgPoints(),
    img2cam_converter= Img2CamCoords(),
    z_clipper=ZClipper([0.2, 1.8]),
)

In [ ]:
### compare number of points in full pc

for dataset in datasets:
    for i in range(len(dataset)):
        sample = dataset[i]

        full_pc, segmented_pc = preprocessor(sample)
